<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 200px; display: inline" alt="IMT"/> </a>
</center>

# IA Framework.
## Lab 1  - Introduction to Pyspark.
#### Part 1 Data munging with <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/> </a>

**Resume**: The objective of this notebook is to discover [Spark](https://spark.apache.org/) framework and its python API [`PySpark`](http://spark.apache.org/docs/latest/api/python/). 
We will see the main motivation to use these frameworks that allow to apply distributed task on clusters and understand the concept of **RDD**(*Resilient Distributed Datasets*), main abstraction Spark provides , and how we can use them.

## Introduction

### When using Spark ?

When data becomes to big for either RAM or Disk memory and/or where computation time are to high for your computer.  
**Spark** allow you do parallelize your tasks to different clusters and provides an interface to do it easily. 

Machine learning algorithm (supervised or unsupervised) are iterative algorithm. Using them with *Hadoop* technology requires to read and write at each iteration on disk which will make learning really slow. [Spark](http://spark.apache.org/)' (*Resilient Distributed Dataset* or **RDD** [Zaharia et al. 2012](https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf). We won't get deeply intoo Hadoop details. See [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document) for an introduction to *MapReduce*, its limitation. 

Spark allow to stock data in each cluster trought **RDD** and use them only in read-only mode which allow faster run of different algorithms.


#### About spark

* **Spark** can be used trough *Java*, *Scala*, *R* and *Python* API. We will see  [`PySpark`](http://spark.apache.org/docs/latest/api/python/) API all along this TP. 
* **Spark** has four main librairies:
 
    * [`SparlSQL`](https://spark.apache.org/docs/latest/sql-programming-guide.html) which allows to access really big and various kind of data, structred or not by executing SQL syntax. (Part 3).
    * [`MLlib`](http://spark.apache.org/docs/latest/ml-guide.html) which contains statistical and machine learning algorithm (Part 2 and 4).
    * [`SparkStreaming`](https://spark.apache.org/docs/latest/streaming-programming-guide.html) for live data stream processing (No cover in this TP).
    * [`GraphX`](https://spark.apache.org/docs/latest/graphx-programming-guide.html) for graphs and graph-parallel computation (No cover in this TP).



#### Spark locally
**Spark** is designed to run on cluster to take advantage of it.
In this TP we will run Spark locally to understand how to use *Pyspark* API, but we won't be able to realize the all potential of Spark.


#### Warnings

For the training phase of a machine learning model estimation, it's often easier to use bigger ressources (more Ram and CPU), than using **Spark** on distributed cluster. Using **Spark** (or **Hadoop**) will be more efficient for extracting sampling, preprocessing the data (See *Cdiscount* application in [Besse et al. 2016](https://hal.archives-ouvertes.fr/hal-01350099v2/document). 


#### References
Official [documentation](https://spark.apache.org/docs/latest/). [Karau et al. (2015)](http://index-of.co.uk/Big-Data-Technologies/Learning%20Spark%20%20Lightning-Fast%20Big%20Data%20Analysis%20.pdf).


## Configuration

### Configuration

**Spark** needs a configuration to be used. We create for that a `SparkConf` object that contains information about your application (where is the master, the cluster, etc..) and  `SparkContext` object which tell us how to access a cluster.

Here, these objects have already been defined within the pyspark kernel of your notebook, and can be directly used:

In [1]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

## Dataset
This notebook has been inspired by the one build by [J. A. Dianes](https://github.com/jadianes/spark-py-notebooks) which use the dataset used in the  [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html) competition. It contains 9M of interactions within a newtork (see description [here](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names)). The objective of this contest was to detect attack in a networf from various features build or computed on transaction or interaction with this network.

In [2]:
import urllib.request
# Download the file from URL
DATA_PATH=""  #<-- Put here where you want to stock the file.
f = urllib.request.urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz",DATA_PATH+"kddcup.data_10_percent.gz")

## RDD

### Create a RDD

#### From  a python object
You can convert a python object to a RDD with the `parallelize` function.

In [3]:
l = range(100)
l

range(0, 100)

In [6]:
data = sc.parallelize(l)
data

PythonRDD[1] at RDD at PythonRDD.scala:53

#### From a file
The other way is to read a file from your computer as a RDD with the `textfile` function. You will read here the dataset previously defined.

In [7]:
data_file = DATA_PATH+"kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
raw_data.take(2)

['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.',
 '0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,19,19,1.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00,normal.']

### **Transformation** and **Actions** on RDD.

They are two type of operations you can apply on RDD **Transformation** and **Actions**. 

* **Transformation** allow to create a new RDD from an existing one.
* **Actions** apply computation on the RDD and return a value (like the .count function)

#### Map

The `.map` function is one of the simplest **Transformation**. It allows to apply the same function to each entries of the rdd.
Below we apply a function to split each string entry of the RDD to get a list of elements :

In [8]:
import time
ts = time.time()
csv_data = raw_data.map(lambda x : x.split(","))
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

Time execution :0.0019 seconds


We can now observe the new RDD produced by the transformation : 

#### Take
The `.take` function is one of the simpliest action. It convert the rdd to a python list of n element.

In [9]:
import time
ts = time.time()
csv_take  =csv_data.take(2)
print(csv_take)
te =time.time()
print("Time execution :%.4f seconds" %(te-ts))

[['0', 'tcp', 'http', 'SF', '181', '5450', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '9', '9', '1.00', '0.00', '0.11', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.'], ['0', 'tcp', 'http', 'SF', '239', '486', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '8', '8', '0.00', '0.00', '0.00', '0.00', '1.00', '0.00', '0.00', '19', '19', '1.00', '0.00', '0.05', '0.00', '0.00', '0.00', '0.00', '0.00', 'normal.']]
Time execution :0.0488 seconds


**Question** 
* What can you say about the time execution fo the two cells above? Is it normal? Why?

#### More complex map function

On the above cell, the function apply on each entry is defined within the map function thanks to the `lambda` operator. 

You can also define first a python function and then apply on the RDD. This will allow you to define more complex and more readable function.

Below is a function that convert element to float, if the string can be converted to a float, and separe features from the label.

In [10]:
# Fontion qui sépare les champ (elems=valeur) et extrait la 41ème = clef.
def parse_interaction(l):
    elems = l
    features =[]
    for e in  elems[:-1]:
        try:
            e=float(e)
        except ValueError:
            e=e
        features.append(e)
    y = elems[-1]
    return (features, y)
# Affichage des 5 premiers
key_csv_data = csv_data.map(parse_interaction)
print(key_csv_data.take(1))

[([0.0, 'tcp', 'http', 'SF', 181.0, 5450.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 8.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 9.0, 9.0, 1.0, 0.0, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0], 'normal.')]


#### Count

The `count` function is a **actions** which allow to count number of elements on a rdd. 

Let's compute the number of element in the three RDD created so far.

In [11]:
ts = time.time()
print(raw_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))


ts = time.time()
print(csv_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

ts = time.time()
print(key_csv_data.count())
te= time.time()
print("Time execution :%.4f seconds" %(te-ts))

494021
Time execution :0.4217 seconds
494021
Time execution :0.9110 seconds
494021
Time execution :4.9017 seconds


**Question** 
* What can you say about the results of the three application of the count function?
* What can you say about time execution?

#### Distinct
the `distinct` function is a **transformation** that build a RDD where all duplicated element are remove

In the cell below we build a list of all protocol in the dataset:

In [12]:
protocol = csv_data.map(lambda x : x[1]).distinct()
protocol.take(10)

['tcp', 'udp', 'icmp']

#### Filter
Another very used **Transformation** is the `filter` function. It wil create a smaller dataset based on a custom condition function.

The cell below, will build a smaller RDD which contains only '.normal' entries.

In [13]:
normal_csv_data = csv_data.filter(lambda x: 'normal.' == x[-1])

In [14]:
t0 = time.time()
normal_count = normal_csv_data.count()
total_count = csv_data.count()
tt = time.time() - t0
print("They are %d normal interactions (over %d)" %(normal_count, total_count))

They are 97278 normal interactions (over 494021)


#### Collect
`collect` is an ***action** similare to `.take` **action**,  except that it will convert the ALL dataset to a python list.

In [15]:
t0 = time.time()
all_raw_data = raw_data.collect()
tt = time.time() - t0
print("Data collected in %.3f seconds" %tt)
all_raw_data[0]

Data collected in 2.292 seconds


'0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.'

In [16]:
t0 = time.time()
print(len(all_raw_data))
tt = time.time() - t0
print("Time runing : %.3f" %tt)

494021
Time runing : 0.000


#### Combining transformation

You've seen above that transformation are *Lazy* Operators, combining **transformations** on various RDD are also *Lazy*. The all pipeline will be run only when a **action** is applied.

In [17]:
t0 = time.time()
data_file = DATA_PATH+"kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)
key_csv_data = raw_data.map(parse_interaction)
normal_key_interactions = key_csv_data.filter(lambda x: x[0] == "normal.")
tt = time.time() - t0
print("Time runing : %.3f" %tt)

Time runing : 0.019


In [18]:
t0 = time.time()
count_normal_interaction= normal_key_interactions.count()
tt = time.time() - t0
print("Time runing : %.3f" %tt)
print("There are %d 'normal' interactions" %count_normal_interaction)

Time runing : 25.913
There are 0 'normal' interactions


## Operation on rdd.
Operation are **Transformation** that can by apply on different RDD. 

#### Substract
`Substract` operation enable to create a RDD *C* from RDDs *A* and *B* by taking all entries from *A* that are not in *B*.

The cells below allow to create a RDD with no normal data in it.

In [19]:
normal_raw_data = raw_data.filter(lambda x: "normal." in x) #<- RDD with normal data
attack_raw_data = raw_data.subtract(normal_raw_data) #<-- RDD with 'anormal' data.

In [20]:
t0 = time.time()
raw_data_count = raw_data.count()
tt = time.time() - t0
print("All count in {} secs".format(round(tt,3)))

All count in 0.397 secs


In [21]:
t0 = time.time()
normal_raw_data_count = normal_raw_data.count()
tt = time.time() - t0
print("Normal count in {} secs".format(round(tt,3)))

Normal count in 0.464 secs


In [22]:
t0 = time.time()
attack_raw_data_count = attack_raw_data.count()
tt = time.time() - t0
print("Attack count in {} secs".format(round(tt,3)))

Attack count in 2.01 secs


In [23]:
print("Il y a {} interactions normales et {} attaques, pour un total de {} interactions".format(normal_raw_data_count,attack_raw_data_count,raw_data_count))

Il y a 97278 interactions normales et 396743 attaques, pour un total de 494021 interactions


#### cartesian

The cartesian product (`.cartesian`) returns every possible pair between elements of two rdd. (To be used with caution if RDDs are really big)

**Exercise**
Write function do display all possible pair Protocol (Second columns) and Services (Third column) within the dataset using  `.cartesian` function.

In [24]:
# %load solutions/exercise1_1.py

### More **Actions** on RDDs
So far we have used only *native* action function (count, distinct, take etc..) We will now define custom action with `reduce` and `aggregate` function.

#### reduce

`reduce` function take a function as an argument that will describe how elements from the RDD are combined.

The code below will build two rdd containings duration time from normal and attack data

In [26]:
normal_csv_data =csv_data.filter(lambda x : x[-1]=="normal.")
attack_csv_data =csv_data.filter(lambda x : x[-1]!="normal.")

normal_duration_data = normal_csv_data.map(lambda x: int(x[0]))
attack_duration_data = attack_csv_data.map(lambda x: int(x[0]))

We use them `reduce` function to compute the total duration of all these actions.

In [30]:
total_normal_duration = normal_duration_data.reduce(lambda x, y: x + y)
total_attack_duration = attack_duration_data.reduce(lambda x, y: x + y)

print("Total duration for 'normal' interactions is {}".\
    format(total_normal_duration))
print("Total duration for 'attack' interactions is {}".\
    format(total_attack_duration))

Total duration for 'normal' interactions is 21075991
Total duration for 'attack' interactions is 2626792


And we compute the means

In [31]:
normal_count = normal_duration_data.count()
attack_count = attack_duration_data.count()

print("Mean duration for 'normal' interactions is {}".\
    format(round(total_normal_duration/float(normal_count),3)))
print("Mean duration for 'attack' interactions is {}".\
    format(round(total_attack_duration/float(attack_count),3)))

Mean duration for 'normal' interactions is 216.657
Mean duration for 'attack' interactions is 6.621


The code above looks quite complicated for such a simple operation. For example the reduce function does not allow to define a "start" value, like in Python. 

**Question** : How would you implement the `count` function with `reduce` function in Python?  Why this can't be applied directly on Spark?

In [34]:
#%load solutions/exercise1_2.py


#### aggregate

The `aggregate` function allows to overcome the problem. The function takes as arguement : 
1. The initialization
2. A function that describes how element of the rdd are combined
3. A function that describes how element from different cluster will be combined

In [42]:
normal_sum_duration = normal_duration_data.aggregate(0, # valeurs initiales à 0
    lambda acc, value: acc + value, # Somme des durées et cumul des interactions
    lambda acc1, acc2: acc1+ acc2 # cumul des accumualteurs
    )
normal_sum_duration

21075991

The aggregate function could be used to compute both total sum of duration and count of element in one call of the function.

**Exercise** Write a function that return total duraction of normal attack AND the count of normal attack

In [43]:
attack_sum_count = attack_duration_data.aggregate(
    (0,0), # the initial value
    #todo;
    #todo
    )

print("Durée moyenne des interactions agressives {}".\
    format(round(attack_sum_count[0]/float(attack_sum_count[1]),3)))

Durée moyenne des interactions agressives 6.621


In [ ]:
#% load solutions/exercise1_3.py

### ReducebyKey & CombineByKey

`ReducebyKey` & `CombineByKey` are function that allow to apply reduce or aggregate function by key (which is the base of MapReduce function.

We need for that, to define a RDD where for each entry, the first element is the key, and the second element is the value.

In [53]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) # x[41] contient le type normal ou non
key_value_duration.take(1)

[('normal.', 0.0)]

#### Reducee By Key


The `reduceByKey` function is the used to apply reduce function for each key on the first columns.


In [55]:
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)
durations_by_key.collect()

[('normal.', 21075991.0),
 ('buffer_overflow.', 2751.0),
 ('loadmodule.', 326.0),
 ('perl.', 124.0),
 ('neptune.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('pod.', 0.0),
 ('teardrop.', 0.0),
 ('portsweep.', 1991911.0),
 ('ipsweep.', 43.0),
 ('land.', 0.0),
 ('ftp_write.', 259.0),
 ('back.', 284.0),
 ('imap.', 72.0),
 ('satan.', 64.0),
 ('phf.', 18.0),
 ('nmap.', 0.0),
 ('multihop.', 1288.0),
 ('warezmaster.', 301.0),
 ('warezclient.', 627563.0),
 ('spy.', 636.0),
 ('rootkit.', 1008.0)]

We note that `reduceByKey` is a **Transformation**

In [48]:
counts_by_key = key_value_data.countByKey()
counts_by_key

defaultdict(int,
            {'normal.': 97278,
             'buffer_overflow.': 30,
             'loadmodule.': 9,
             'perl.': 3,
             'neptune.': 107201,
             'smurf.': 280790,
             'guess_passwd.': 53,
             'pod.': 264,
             'teardrop.': 979,
             'portsweep.': 1040,
             'ipsweep.': 1247,
             'land.': 21,
             'ftp_write.': 8,
             'back.': 2203,
             'imap.': 12,
             'satan.': 1589,
             'phf.': 4,
             'nmap.': 231,
             'multihop.': 7,
             'warezmaster.': 20,
             'warezclient.': 1020,
             'spy.': 2,
             'rootkit.': 10})

#### combineByKey

`combineByKey` if for `aggregate` whate `reduceByKey` is to `reduce` function.


In [58]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # valeur initiale x and compteur 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # Combiner une paire  avec une paires d'accumulateurs (somme et incrément)
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combinaison des accumulateurs
     )
sum_counts.collect()

[('normal.', (21075991.0, 97278)),
 ('buffer_overflow.', (2751.0, 30)),
 ('loadmodule.', (326.0, 9)),
 ('perl.', (124.0, 3)),
 ('neptune.', (0.0, 107201)),
 ('smurf.', (0.0, 280790)),
 ('guess_passwd.', (144.0, 53)),
 ('pod.', (0.0, 264)),
 ('teardrop.', (0.0, 979)),
 ('portsweep.', (1991911.0, 1040)),
 ('ipsweep.', (43.0, 1247)),
 ('land.', (0.0, 21)),
 ('ftp_write.', (259.0, 8)),
 ('back.', (284.0, 2203)),
 ('imap.', (72.0, 12)),
 ('satan.', (64.0, 1589)),
 ('phf.', (18.0, 4)),
 ('nmap.', (0.0, 231)),
 ('multihop.', (1288.0, 7)),
 ('warezmaster.', (301.0, 20)),
 ('warezclient.', (627563.0, 1020)),
 ('spy.', (636.0, 2)),
 ('rootkit.', (1008.0, 10))]

In [61]:
duration_means_by_type = sum_counts.map(lambda lambda_args: (lambda_args[0], round(lambda_args[1][0]/lambda_args[1][1],3)))
duration_means_by_type.collect()

[('normal.', 216.657),
 ('buffer_overflow.', 91.7),
 ('loadmodule.', 36.222),
 ('perl.', 41.333),
 ('neptune.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 2.717),
 ('pod.', 0.0),
 ('teardrop.', 0.0),
 ('portsweep.', 1915.299),
 ('ipsweep.', 0.034),
 ('land.', 0.0),
 ('ftp_write.', 32.375),
 ('back.', 0.129),
 ('imap.', 6.0),
 ('satan.', 0.04),
 ('phf.', 4.5),
 ('nmap.', 0.0),
 ('multihop.', 184.0),
 ('warezmaster.', 15.05),
 ('warezclient.', 615.258),
 ('spy.', 318.0),
 ('rootkit.', 100.8)]